# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [23]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

#import holoviews as hv
#import cartopy
#import cartopy.crs as ccrs
#import geoviews as gv # noqa
#import pyproj

# Import API key
from challenge6_api_keys import geoapify_key

In [24]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head(1000)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,megion,61.0296,76.1136,-9.99,97,99,3.72,RU,1676524985
1,1,port alfred,-33.5906,26.8910,19.54,96,99,0.45,ZA,1676524900
2,2,norman wells,65.2820,-126.8329,-27.98,76,75,1.54,CA,1676524979
3,3,ancud,-41.8697,-73.8203,8.61,72,62,3.98,CL,1676524896
4,4,tocopilla,-22.0920,-70.1979,21.61,84,37,1.35,CL,1676524914
...,...,...,...,...,...,...,...,...,...,...
536,536,the valley,18.2170,-63.0578,25.05,77,2,9.56,AI,1676525001
537,537,monrovia,6.3005,-10.7969,24.97,91,16,0.76,LR,1676525080
538,538,whitefish,48.4111,-114.3376,-10.14,92,0,1.54,US,1676525187
539,539,chum phae,16.5443,102.0992,23.05,54,100,2.76,TH,1676525188


In [25]:
# Reviewing data types
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

In [26]:
# Reviewing data counts
city_data_df.count()

City_ID       541
City          541
Lat           541
Lng           541
Max Temp      541
Humidity      541
Cloudiness    541
Wind Speed    541
Country       537
Date          541
dtype: int64

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [27]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points('Lng', 'Lat', geo=True, color='red', alpha=0.5, 
                                        xlim=(-200, 200), ylim=(-70, 70), tiles='OSM')

# Display the map plot_1
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [28]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df[(city_data_df["Max Temp"] < 29) &
                             (city_data_df["Max Temp"] > 16) &
                             (city_data_df["Wind Speed"] < 10) &
                             (city_data_df["Cloudiness"] < 15) &
                             (city_data_df["Humidity"] < 30)
                            ] 
# Drop any rows with null values
ideal_city_df=ideal_city_df.dropna()

# Display sample data
ideal_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
55,55,atar,20.5169,-13.0499,20.53,26,3,4.11,MR,1676524986
64,64,babai,22.7000,77.9333,27.86,17,0,1.32,IN,1676524940
130,130,dalbandin,28.8947,64.4101,23.58,11,6,2.90,PK,1676524974
138,138,jumla,29.2747,82.1838,16.54,16,0,2.34,NP,1676525033
143,143,guna,24.6500,77.3167,26.20,19,0,2.77,IN,1676525034


### Step 3: Create a new DataFrame called `hotel_df`.

In [29]:
coordinates_df = ideal_city_df["Lat"] + ideal_city_df["Lng"]

In [30]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
55,atar,MR,20.5169,-13.0499,26,
64,babai,IN,22.7000,77.9333,17,
130,dalbandin,PK,28.8947,64.4101,11,
138,jumla,NP,29.2747,82.1838,16,
143,guna,IN,24.6500,77.3167,19,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [31]:
# Set parameters to search for a hotel
Lat = hotel_df["Lat"]
Lng = hotel_df["Lng"]
radius = 1000000
filters = f"circle:{Lng},{Lat},{radius}"
bias = f"proximity:{Lng},{Lat}"
params = {"radius": 1000000,
          "type": "hotel",
          "key": geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = filters
    params["bias"] = bias
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
atar - nearest hotel: No hotel found
babai - nearest hotel: No hotel found
dalbandin - nearest hotel: No hotel found
jumla - nearest hotel: No hotel found
guna - nearest hotel: No hotel found
abu dhabi - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
dali - nearest hotel: No hotel found
ibra - nearest hotel: No hotel found
raga - nearest hotel: No hotel found
boromo - nearest hotel: No hotel found
pokhara - nearest hotel: No hotel found
savalou - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
55,atar,MR,20.5169,-13.0499,26,No hotel found
64,babai,IN,22.7000,77.9333,17,No hotel found
130,dalbandin,PK,28.8947,64.4101,11,No hotel found
138,jumla,NP,29.2747,82.1838,16,No hotel found
143,guna,IN,24.6500,77.3167,19,No hotel found
153,abu dhabi,AE,24.4667,54.3667,29,No hotel found
183,tura,IN,25.5198,90.2201,20,No hotel found
321,dali,CN,25.7000,100.1833,21,No hotel found
327,ibra,OM,22.6906,58.5334,15,No hotel found
358,raga,SS,8.4596,25.6780,23,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [32]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points('Lng', 'Lat', geo=True, color='red', alpha=0.5, 
                                        xlim=(-200, 200), ylim=(-70, 70), tiles='OSM')

# Display the map plot_1
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]